In [ ]:
# import libraries

import skimage
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, morphology, measure, feature, draw
import matplotlib.pyplot as plt
from support.bounding_boxes import draw_bounding_boxes, create_bounding_boxes
# capture image with picam2
from picamera2 import PiCamera2
camera = PiCamera2()

In [ ]:
camera.start()
image = camera.capture_array()
camera.stop()
# take green channel
image = image[:,:,1]
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
# configure parameters

t_rect_area = 200
t_small_objects = 75
canny_sigma = 0.5
dilation_size = 25
median_filter_size = 3

plt.imshow(image > 0.2, cmap='gray')
plt.show()

# resize image
img = skimage.transform.resize(image, (300, 350))
plt.imshow(img, cmap='gray')
plt.show()

# use median filter
img = filters.median(img, np.ones((median_filter_size, median_filter_size)))
plt.imshow(img, cmap='gray')
plt.show()

# capture edges with canny edge detector
edges = feature.canny(img, canny_sigma)
plt.imshow(edges, cmap='gray')
plt.show()

# dilate image
dilated = morphology.dilation(edges, np.ones((3,3)))
plt.imshow(dilated, cmap='gray')
plt.show()

# use flooding algorithm to label connected components
labels = measure.label(np.logical_not(dilated))
plt.imshow(labels, cmap='tab20')
plt.show()

# remove small objects
labels = morphology.remove_small_objects(labels, t_small_objects)
rectangles = create_bounding_boxes(labels)
draw_bounding_boxes(rectangles)
plt.imshow(labels, cmap='tab20')
plt.show()

# remove all small rectangles
rectangles = [rect for rect in rectangles if rect[2] * rect[3] > t_rect_area]
draw_bounding_boxes(rectangles)
plt.imshow(img, cmap='gray')
plt.show()